In [ ]:
#r "nuget: TorchSharp-cpu"

open TorchSharp
open type TorchSharp.TensorExtensionMethods

open Microsoft.DotNet.Interactive.Formatting
Formatter.SetPreferredMimeTypesFor(typeof<torch.Tensor>, "text/plain")
Formatter.Register<torch.Tensor>(fun (x:torch.Tensor) -> x.ToString(true))

# Tensors

In TorchSharp, as in all deep learning, the fundamental data type is a 'tensor,' which is simply a generalized matrix. In Linear Algebra, a one-dimensional arrays are called 'vector,' and a two-dimensional array is a 'matrix.' Generalizing on that, a tensor is simply an N-dimensional array. 

Please note that there is an overloaded use of the word 'dimensions' here -- in physics, a vector (one dimension) with three elements is used to represents a point in space, one element for each spatial dimension. When we speak of 'dimension' in these tutorials, it is the number of tensor dimensions that is of interest.

So, let's get started with tensors by creating some.

## Constant-Filled Tensors

The simplest tensor creation primitives just initializes a tensor with either 0 or 1 in all its elements. The arguments passed in is the size of each dimension. Think of the first dimension as the rows of a table, the second as the columns, and then you just have to generalize things in your head after that. In the examples below, we'll mostly be creating 3x4 matrices, for simplicity.

One thing to note is that .NET Interactive will show the object and its fields, etc. when you say just 't' at the end of the notebook cell. What we want for tensors is to show the contents, and there's a special version of ToString() taking a Boolean that shows not just the size and type of the tensor, but also its contents. The special .NET Interactive formatter that is found at the top of the notebook (and should be at the top of all notebooks using TorchSharp) uses TorString(true).

In [ ]:
let t = torch.ones(3, 4)
t

If you have more than two dimensions, the special version of ToString(t) will try to format it in a way that makes sense to a human:

In [ ]:
torch.ones(2,4,4)

If you intend to fill the tensor with values from somewhere else, in other words because you pre-allocated it, there's an 'empty' factory that is faster than using anything else. The values are just whatever was found in memory when the tensor was created. Don't mistake that for random values, though -- they won't adhere to any particular distribution. In fact, one of the main uses for empty tensors is to create one, and then fill it with random numbers from some particular distribution.

In [ ]:
torch.empty(4,4)

You can also create a tensor from any value you want. F# doesn't yet support implicit conversions, to for the time being, we have to explicitly convert some operands.

In [ ]:
torch.full(4,4,3.14f.ToScalar())

Sometimes, you want to display more than one value in a cell. To do that, you need to rely on good-old-fashioned printing:

In [ ]:
Console.Write(torch.zeros(4,4).ToString(true));
Console.Write(torch.ones(4,4).ToString(true));

All that typing will get pretty tedious, so we added an extension method `print()` that is much quicker to type. It really should only be used in notebooks.

`print()` returns the tensor it's invoked on. In C#, adding a ';' to the end of the last line supresses the automatic printing of the last value, but the same isn't true for F#. In order to avoid printing the last expression, you have to send it to `ignore()`

In [ ]:
// This doesn't work yet -- wait for the next release:
torch.zeros(4,4).print()
torch.ones(4,4).print() |> ignore

You may have noticed that each tensor has a 'type = Float32' attribute. This is a peculiarity about the TorchSharp tensor type -- the element type does not show up in the type, Tensor is not Tensor\<T\>. This is so because the underlying C++ / CUDA runtime represents tensors this way, and it makes it easier to port code from Python, too.

You may also have noticed that tensors are created using factories, not constructors. Also, the naming convention doesn't look anything like .NET. We chose to step away from .NET conventions in order to make it easier to port code from Python. We know this will upset some, and please some, but it's the decision we came to after a long time of deliberating.

Anyway, 'Float32' is the default, but you can create tensors of other types, too, including complex tensors:

In [ ]:
torch.zeros(4,4, dtype=torch.int32)

In [ ]:
torch.zeros(4, 4, dtype=torch.complex64)

To access the contents of a tensor, you treat it as a multi-dimensional array (note that the number of dimensions also isn't part of the type itself). When you do, you'll be surprised to see that the result of the indexing operator is another tensor, one that has no shape -- this is how TorchSharp represents a scalar value. Later in this tutorial, we will see why. For now, just know that you have to extract the value using a function, based on the type you expect to get out.

In PyTorch, there's a method '.item()' used for this purpose. In TorchSharp, it's a templatized method: 

In [ ]:
let t = torch.zeros(4,4, dtype=torch.int32)
t.[0L,0L].print()
t.[0L,0L].item<int>()

To write to a single element, you have to create a tensor from the value you want to write.

In [ ]:
t.[0L,0L] <- torch.tensor(35)
t

## Randomized Tensors

In machine learning, random number generation is very important, and you often end up using the RNG APIs to create tensors. There are a big number of RNGs, most of them for floating point values, but there are some for integers, too.

The usual suspects are present -- normal and uniform distributions, binomial (true/false or 0/1) and uniformly distributed integers. A separate tutorial will disscuss random number generation in more detail.

In [ ]:
// Normal distribution
torch.randn(3L,4L)

In [ ]:
// Uniform distribution between [0,1[
torch.rand(3,4)

In [ ]:
// Uniform distribution between [100,110[
(torch.rand(3,4) * torch.tensor(10) + torch.tensor(100))

# Ranges

Another common and convenient tensor factory is `arange()`, which is used to create a 1D tensor with numbers ranging from a min to a max, exclusive of the max. You can provide the step value, or let it be the default, which is 1:

In [ ]:
torch.arange(3L.ToScalar(),14L.ToScalar())

As you can see, `arange` behaves a little differently from other factories -- the default element type is not float32, it's inferred from the arguments. To use floating-point, we an either specify the type, or use floating-point values as arguments:

In [ ]:
torch.arange(3.0f.ToScalar(), 14.0f.ToScalar())

The step argument will allow us to get a more fine-grained number series:

In [ ]:
torch.arange(3.0f.ToScalar(), 5.0f.ToScalar(), step=0.1f.ToScalar())

There is no way to make `arange` produce anything but a 1D tensor, so a common thing to do is to reshape it as soon as it's created:

In [ ]:
torch.arange(3.0f.ToScalar(), 5.0f.ToScalar(), step=0.1f.ToScalar()).reshape(4L,5L)

When printing out floating-point tensors, it is often helpful to provide a formatting string, applied only to FP elements, including the real and imaginary parts of complex numbers. `.ToString(true, "0.00")` would be a lot to type every time, so TorcSharp defines a shorter (and Python-looking) method `str()` that does the trick. `print()` also takes a format string.

In [ ]:
torch.arange(3.0f.ToScalar(), 5.0f.ToScalar(), step=0.1f.ToScalar()).reshape(4L,5L).str("0.00")

# Common Tensor Properties and Operations

There is a large number of operations on tensors. Here are some of the most commonly used ones.

In [ ]:
let t = torch.arange(3.0f.ToScalar(), 5.0f.ToScalar(), step=0.1f.ToScalar()).reshape(2L,2L,5L);

// The overall shape of the tensor:
t.shape

In [ ]:
// The number of dimensions:
t.ndim

In [ ]:
// The total number of elements held in the tensor:
t.numel()

In [ ]:
// Move the tensor to the GPU and back to the CPU:

// t.cuda() // Uncomment if you're using one of the CUDA backends.
t.cpu()

In [ ]:
torch.arange(3.0f.ToScalar(), 5.0f.ToScalar(), step=0.1f.ToScalar()).reshape(4L,5L).print()
torch.arange(3.0f.ToScalar(), 5.0f.ToScalar(), step=0.1f.ToScalar()).reshape(4L,5L).T

It's important to note that transposing isn't the same as simply reshaping to a different shape:

In [ ]:
torch.arange(3.0f.ToScalar(), 5.0f.ToScalar(), step=0.1f.ToScalar()).reshape(5L,4L).print()
torch.arange(3.0f.ToScalar(), 5.0f.ToScalar(), step=0.1f.ToScalar()).reshape(4L,5L).T

In [ ]:
// Clone a tensor:
let s = t.clone()
s

TorcgSharp has great support for complex tensors, which are as easy to create as others. Complex numbers are commonly used in signal processing scenarios, such as audio analysis. Complex numbers have two components -- the real and the imaginary parts.

In [ ]:
let ct = torch.rand(3,4,dtype=torch.ScalarType.ComplexFloat32);
ct.str("0.00")

To get the real and imaginary parts, there are property accessors for that:

In [ ]:
ct.real

In [ ]:
ct.imag

When you create random complex tensors, both the real and imaginary parts are random. However, when you create a tensor with `torch.ones`, only the real part is filled in -- the imaginary part is '0'

In [ ]:
torch.ones(3,4,dtype=torch.ScalarType.ComplexFloat32)

Experienced PyTorch users will notice that TorchSharp doesn't represent complex numbers in the str() output exactly the same: TorchSharp uses 'i' instead of 'j' to indicate the imaginary part, and we don't print the real or imaginary part if it is zero (unless they are both zero, of course).